<a href="https://colab.research.google.com/github/RandyV37/Telecom-X-part-2-Data-Science-/blob/main/TelecomX(Challenge_parte_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importar librerias y cargar mi dataset

In [19]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('TelecomX_Data_Tratada.csv')


## Ubicar Variables categoricas

In [20]:
Variables_Categoricas = df.select_dtypes(include=['object']).columns.tolist()

## Encoding

In [21]:
encoder = OneHotEncoder(
    drop='first',
    sparse_output=False,
    handle_unknown='ignore'
)

Array_codificado = encoder.fit_transform(df[Variables_Categoricas])
Variables_categoricas_codificadas = encoder.get_feature_names_out(Variables_Categoricas)
df_x_codificada = pd.DataFrame(Array_codificado, columns=Variables_categoricas_codificadas, index=df.index)
variables_numericas = df.drop(columns=Variables_Categoricas)
Df_final = pd.concat([variables_numericas, df_x_codificada], axis=1)

In [25]:
Df_final.value_counts(subset="Cliente_Detractor")

,count
Cliente_Detractor,
0,5398
1,1869
